In [6]:
! pip freeze | grep ninja

# fruit and vegetable detection for yolov8

In [1]:
# Properly install detectron2. (Please do not install twice in both ways)
! python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-htg20o5h
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-htg20o5h
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.3 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp31

In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.5 ; cuda:  cu124
detectron2: 0.6


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [8]:
#! python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
# (add --user if you don't have permission)

# Or, to install it from a local clone:
#! git clone https://github.com/facebookresearch/detectron2.git
#! python -m pip install -e detectron2

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-vgf8g9sw
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-vgf8g9sw
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.0 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp31

In [4]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [5]:
from google.colab import drive
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

drive.mount('/content/drive')
path_to_drive = "/content/drive/MyDrive/Artefact/Projet_batch_51"
base_dir = "/content/"


locale.getpreferredencoding = getpreferredencoding

Mounted at /content/drive


In [6]:
! cd /content
! #rm -r sample_data/
! rm /content/sample_data/*
! sleep 3
! cp /content/drive/MyDrive/Artefact/Projet_batch_51/input_data/sample_data/* /content/sample_data/


In [59]:
import cv2
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Load an image
image_path = "/content/sample_data/Royal_gala_2.jpg"
image = cv2.imread(image_path)

# Create Detectron2 config
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

# Create predictor
predictor = DefaultPredictor(cfg)

# Perform prediction
outputs = predictor(image)

[02/20 16:41:46 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl ...


roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.mask_head.mask_fcn2.{bias, weight}
roi_heads.mask_head.mask_fcn3.{bias, weight}
roi_heads.mask_head.mask_fcn4.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [60]:
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

tensor([47, 60], device='cuda:0')
Boxes(tensor([[ 623.3099, 1151.2563, 2080.0232, 2526.4651],
        [   0.0000,   13.6696, 3000.0000, 2948.8921]], device='cuda:0'))


In [61]:
print(outputs)

{'instances': Instances(num_instances=2, image_height=3000, image_width=3000, fields=[pred_boxes: Boxes(tensor([[ 623.3099, 1151.2563, 2080.0232, 2526.4651],
        [   0.0000,   13.6696, 3000.0000, 2948.8921]], device='cuda:0')), scores: tensor([0.9881, 0.8002], device='cuda:0'), pred_classes: tensor([47, 60], device='cuda:0'), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ...,

In [62]:
outputs["instances"].pred_classes[0]

tensor(47, device='cuda:0')

In [63]:
MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).get("thing_classes")[60]

'dining table'

In [64]:
MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).get("thing_classes")[outputs["instances"].pred_classes[0]]

'apple'

dict_keys(['instances'])

In [65]:
for image in lst_file_input:
  im = cv2.imread(image)
  outputs_v2 = predictor(im)
  detected_classes = [MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).get("thing_classes")[i] for i in outputs_v2["instances"].pred_classes]
  score = [score for score in outputs_v2["instances"].scores]
  print(image,str(detected_classes),score)


/content/sample_data/Royal_gala_2.jpg ['apple', 'dining table'] [tensor(0.9881, device='cuda:0'), tensor(0.8002, device='cuda:0')]
/content/sample_data/poivron_vert_bag_3.jpg ['bed'] [tensor(0.5837, device='cuda:0')]
/content/sample_data/poivron_vert_bag_2.jpg [] []
/content/sample_data/Pink_Lady_1.jpg ['apple', 'dining table', 'chair'] [tensor(0.9903, device='cuda:0'), tensor(0.7214, device='cuda:0'), tensor(0.6493, device='cuda:0')]
/content/sample_data/Royal_gala1.jpg ['apple', 'dining table'] [tensor(0.9234, device='cuda:0'), tensor(0.6170, device='cuda:0')]
/content/sample_data/Royal_gala_3.jpg ['apple', 'dining table'] [tensor(0.9949, device='cuda:0'), tensor(0.7745, device='cuda:0')]
/content/sample_data/1739108130215.jpg ['apple', 'dining table'] [tensor(0.9905, device='cuda:0'), tensor(0.8365, device='cuda:0')]
/content/sample_data/poivron_rouge_bag_1.jpg [] []
/content/sample_data/poivron_rouge_bag_2.jpg [] []
/content/sample_data/golden_1.jpg ['apple', 'dining table'] [tenso